In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving VMD_Cleaned.xlsx to VMD_Cleaned.xlsx


In [ ]:
df = pd.read_excel("VMD_Cleaned.xlsx")

In [ ]:
# Assuming your DataFrame is named `df`
# Extract the target column (assuming the last column is the target)
target_column = df.columns[-1]  # Get the name of the last column
flux_values = df[target_column]

# Calculate the minimum, maximum, and range values
min_flux = flux_values.min()
max_flux = flux_values.max()
range_flux = max_flux - min_flux

# Print the results
print("Minimum Flux:", min_flux)
print("Maximum Flux:", max_flux)
print("Range of Flux:", range_flux)


Minimum Flux: 6.9
Maximum Flux: 15.8
Range of Flux: 8.9


In [ ]:
df

,T©,P(kPa),concentration(M),Flow Rate(L/h),flux(l/m^2h)
0,319.2,5.1,1.0,50,13.5
1,319.2,5.1,1.1,50,13.4
2,319.2,5.1,1.2,50,13.2
3,319.2,5.1,1.3,50,13.2
4,319.2,5.1,1.4,50,13.1
...,...,...,...,...,...
109,310.2,4.0,2.4,100,8.3
110,310.2,4.0,2.5,100,8.1
111,310.2,4.0,2.7,100,8.0
112,310.2,4.0,2.8,100,7.9


In [ ]:
# Summary statistics for numerical columns (range, mean, etc.)
numerical_summary = df.describe().T
numerical_summary['Range'] = numerical_summary['max'] - numerical_summary['min']
numerical_summary = numerical_summary[['min', 'max', 'mean', 'std', 'Range']]
print("Numerical Summary:\n", numerical_summary)

# Unique levels for each column (useful for categorical features)
unique_counts = df.nunique()
print("\nUnique Levels in Each Column:\n", unique_counts)

# Data types to identify categorical and numerical features
print("\nData Types:\n", df.dtypes)

# Distribution analysis for numerical features
for column in df.select_dtypes(include=['float64', 'int64']).columns:
    print(f"\n{column} Value Counts:\n", df[column].value_counts(bins=5))  # Binned count to view ranges

# Count of unique values for each categorical feature
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\n{column} Levels:\n", df[column].value_counts())

Numerical Summary:
                     min    max        mean        std  Range
T©                310.2  321.9  315.150000   4.366126   11.7
P(kPa)              3.9    6.0    4.733333   0.728153    2.1
concentration(M)    1.0    3.0    1.921053   0.583470    2.0
Flow Rate(L/h)     50.0  100.0   75.000000  25.110376   50.0
flux(l/m^2h)        6.9   15.8   10.498246   2.523973    8.9

Unique Levels in Each Column:
 T©                   5
P(kPa)               6
concentration(M)    19
Flow Rate(L/h)       2
flux(l/m^2h)        67
dtype: int64

Data Types:
 T©                  float64
P(kPa)              float64
concentration(M)    float64
Flow Rate(L/h)        int64
flux(l/m^2h)        float64
dtype: object

T© Value Counts:
 (310.187, 312.54]    38
(312.54, 314.88]     19
(314.88, 317.22]     19
(317.22, 319.56]     19
(319.56, 321.9]      19
Name: count, dtype: int64

P(kPa) Value Counts:
 (3.8970000000000002, 4.32]    38
(4.74, 5.16]                  38
(4.32, 4.74]                  19

In [ ]:
for column in df.columns:
    print(f"Unique values in {column}:\n", df[column].unique(), "\n")

Unique values in T©:
 [319.2 313.7 310.2 321.9 315.7] 

Unique values in P(kPa):
 [5.1 4.4 3.9 6.  5.  4. ] 

Unique values in concentration(M):
 [1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.7 2.8
 3. ] 

Unique values in Flow Rate(L/h):
 [ 50 100] 

Unique values in flux(l/m^2h):
 [13.5 13.4 13.2 13.1 13.  12.8 12.7 12.5 12.4 12.3 12.2 12.1 11.9 11.8
 11.5  9.5  9.4  9.3  9.2  9.1  9.   8.9  8.7  8.6  8.5  8.3  8.1  7.9
  7.8  8.4  8.2  8.   7.7  7.6  7.4  7.1  7.   6.9 15.8 15.5 15.3 15.2
 15.1 14.8 14.7 14.4 14.3 14.2 14.1 14.  13.9 13.8 11.4 11.3 11.1 11.
 10.9 10.8 10.6 10.5 10.4 10.3 10.2 10.1  9.9  9.8  9.6] 



In [ ]:
X = df.iloc[: , : -1]
X= X.to_numpy()
print(X.shape)

y = df.iloc[: , -1]
y = y.to_numpy()
print(y.shape)

(114, 4)
(114,)


In [ ]:
X_train,X_test,y_train,y_test = train_test_split (X,y,test_size = 0.2, random_state = 42)
kf = KFold(n_splits = 5 , shuffle = True , random_state = 42)
mse_list = []
r2_list = []
for train_indices,dev_indices in kf.split(X_train) :
    X_fold_train,X_fold_dev = X[train_indices],X[dev_indices]
    y_fold_train,y_fold_dev = y[train_indices],y[dev_indices]
    scaler = MinMaxScaler()
    X_fold_train = scaler.fit_transform(X_fold_train)
    X_fold_dev = scaler.transform(X_fold_dev)
    model = Sequential([Dense(units = 64 , activation = 'relu',input_shape= (X_fold_train.shape[1],) ),
                       Dense(units = 32 , activation = 'relu'),
                       Dense(units = 16 , activation = 'relu'),
                       Dense(units = 1 , activation = 'linear')
                      ])
    model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))
    model.fit(X_fold_train,y_fold_train, epochs = 500,verbose=0)
    y_fold_dev_pred = model.predict(X_fold_dev)
    mse = mean_squared_error(y_fold_dev,y_fold_dev_pred)
    r2 = r2_score(y_fold_dev,y_fold_dev_pred)
    mse_list.append(mse)
    r2_list.append(r2)
Average_mse = np.mean(mse_list)
Average_r2 = np.mean(r2_list)
print(f" Average mse for all 5 folds = {Average_mse:.3f}")
print(f"Average r2 for all 5 flods = {Average_r2:.3f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
 Average mse for all 5 folds = 0.026
Average r2 for all 5 flods = 0.996


In [ ]:
X_test = scaler.transform(X_test)
y_test_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test,y_test_pred)
r2_test = r2_score(y_test,y_test_pred)
print(f" mse for test dataset is : {mse_test: .3f}")
print(f" r2  for test dataset is : {r2_test: .3f} ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
 mse for test dataset is :  0.081
 r2  for test dataset is :  0.986 


In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 0.142


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Assuming you have the following variables already defined
# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 0.142
Relative MAE for test dataset is: 1.31%


In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (normalize)
scaler = StandardScaler()

# Define the parameter grid for grid search
param_grid = {
    'svr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
    'svr__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 'scale', 'auto'],
    'svr__epsilon': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'pca__n_components': [2, 3, 4, 5]  # Adding PCA components as part of the grid search
}

# Create a pipeline with PCA and SVR
pipeline = Pipeline([
    ('scaler', scaler),
    ('pca', PCA()),
    ('svr', SVR(kernel='rbf'))
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Train the model
grid_search.fit(X_train, y_train)

# Get the best model with optimized parameters
best_model = grid_search.best_estimator_

# Make predictions on training and testing sets
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Evaluate model performance on training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Evaluate model performance on testing set
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Print the results
print("Best Parameters:", grid_search.best_params_)
print("Train Mean Squared Error:", train_mse)
print("Train R2 Score:", train_r2)
print("Test Mean Squared Error:", test_mse)
print("Test R2 Score:", test_r2)

Best Parameters: {'pca__n_components': 3, 'svr__C': 10000, 'svr__epsilon': 0.1, 'svr__gamma': 0.01}
Train Mean Squared Error: 0.005686577904996475
Train R2 Score: 0.9991116750147372
Test Mean Squared Error: 0.002093257832558259
Test R2 Score: 0.9996399501240698


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
2240 fits failed out of a total of 8960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packag

In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 0.036


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Assuming you have the following variables already defined
# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 0.036
Relative MAE for test dataset is: 0.33%


In [ ]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import numpy as np

# Assuming X and y are your features and target variables
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Performing K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_list = []
r2_list = []

# XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Grid search parameters
param_grid = {
    'n_estimators': [100, 200, 300],     # Number of boosting rounds
    'max_depth': [3, 5, 7],               # Maximum depth of a tree
    'learning_rate': [0.1, 0.01, 0.001], # Step size shrinkage to prevent overfitting
    'gamma': [0, 0.1, 0.3],               # Minimum loss reduction required to make a further partition on a leaf node
    'subsample': [0.8, 1.0],              # Fraction of samples used to fit each tree
    'colsample_bytree': [0.8, 1.0]        # Fraction of features used to fit each tree
}

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=kf, scoring='r2', verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters found
best_params = grid_search.best_params_
print("Best parameters found:", best_params)

# Use the best model from grid search
best_xgb = grid_search.best_estimator_

# Predicting on the training set using the best model
y_train_pred = best_xgb.predict(X_train)

# Predicting on the test set using the best model
y_test_pred = best_xgb.predict(X_test)

# Calculating metrics on the training set
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Calculating metrics on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Print results
print(f"Best Parameters: {best_params}")
print(f"Train Mean Squared Error: {mse_train:.3f}")
print(f"Train R2 Score: {r2_train:.3f}")
print(f"Test Mean Squared Error: {mse_test:.3f}")
print(f"Test R2 Score: {r2_test:.3f}")



Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters found: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Best Parameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.8}
Train Mean Squared Error: 0.001
Train R2 Score: 1.000
Test Mean Squared Error: 0.010
Test R2 Score: 0.998


In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE for the test dataset
mae_test = mean_absolute_error(y_test, y_test_pred)

# Print the MAE result
print(f"mae for test dataset is: {mae_test:.3f}")


mae for test dataset is: 0.070


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np

# Assuming you have the following variables already defined
# y_test: actual target values for the test set
# y_test_pred: predicted values for the test set
# mae_test: mean absolute error calculated previously

# Calculate MAE for the test dataset (if not already done)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Calculate the mean of the target values for the Relative MAE
mean_target = np.mean(y_test)

# Calculate Relative MAE for the test dataset
relative_mae_test = (mae_test / mean_target) * 100

# Print results
print(f"MAE for test dataset is: {mae_test:.3f}")
print(f"Relative MAE for test dataset is: {relative_mae_test:.2f}%")


MAE for test dataset is: 0.070
Relative MAE for test dataset is: 0.64%
